In [1]:
import math
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import norm
import statsmodels.api as sm

In [2]:
# Stock class remains unchanged
#S0 current_price
#sigma volatility
#r continuously compunded risk_free_rate

class Stock:
    def __init__(self, s0, sigma, r):
        self.s0 = s0
        self.sigma = sigma
        self.r = r
    
    def __repr__(self):
        return f"Stock(price={self.s0}, volatility={self.sigma}, continuously compunded risk_free_rate={self.r})"


In [3]:
#stockName object of the class stock
#nSteps number discretization steps done
#dt discretization of the grid
class BSModel:
    def __init__(self, stockObj, optObj, numberOfPaths, nSteps):
        self.stockObj = stockObj
        self.opnObj = optObj
        self.numberOfPaths = numberOfPaths
        self.nSteps = nSteps
        
        
    def generateRandonNumbers(self, seed = None):

        if seed is not None:
            np.random.seed(seed)  # Set seed for reproducibility
                    
        # Generate standard normal random numbers
        z = np.random.normal(0, 1, (self.numberOfPaths, self.nSteps))
        
        # Create antithetic variates
        #zAntithetic = -z
        
        # Combine both sets (original and antithetic)
        #randomNumbers = np.vstack((z, zAntithetic))  # Shape: (2 * numberOfPaths, nSteps)

        return z 
    
    def generateSamplePaths(self, randomNumbersMatrix):
        
        #calculate dt
        dt = self.opnObj.timeToExpiry/self.opnObj.nSteps
        
        # Initialize the paths matrix
        samplePaths = np.zeros((self.numberOfPaths, self.opnObj.nSteps + 1))
        samplePaths[:, 0] = self.stockObj.s0  # Set the initial price
        
        #simulation using GBM process
        for t in range(1, self.opnObj.nSteps + 1):
            dW = np.sqrt(dt) * randomNumbersMatrix[:, t - 1]  # Brownian increments
            samplePaths[:, t] = samplePaths[:, t - 1] * np.exp((self.stockObj.r - 0.5 * self.stockObj.sigma**2) * dt + self.stockObj.sigma * dW)
            
        return samplePaths

In [4]:
class Option:
    def __init__(self, timeToExpiry, nSteps, strike):
        self.timeToExpiry = timeToExpiry
        self.nSteps = nSteps
        self.strike = strike
        
class EurOption():

    def __init__(self, stockObj, optionObj):
        self.stockObj = stockObj
        self.optionObj = optionObj
    
    def priceByBSAnalytic(self, optionType):
        
        d1 = (np.log(self.stockObj.s0 / self.optionObj.strike) + (self.stockObj.r + 0.5 * self.stockObj.sigma**2) * (self.optionObj.timeToExpiry)) / (self.stockObj.sigma * np.sqrt(self.optionObj.timeToExpiry))
        d2 = d1 - self.stockObj.sigma * np.sqrt(self.optionObj.timeToExpiry)
        
        if optionType == "call":
            price = self.stockObj.s0 * norm.cdf(d1) - self.optionObj.strike * np.exp(-self.stockObj.r * self.stockObj.T) * norm.cdf(d2)
        elif optionType == "put":
            price = self.optionObj.strike * np.exp(-self.stockObj.r * self.optionObj.timeToExpiry) * norm.cdf(-d2) - self.stockObj.s0 * norm.cdf(-d1)
        else:
            raise ValueError("Invalid option type. Choose 'call' or 'put'.")
    
        return price
    
    def priceByBSMC(self, samplePaths, optionType):
        
        if optionType == "call":
            payoff = np.maximum(samplePaths[:,-1] - self.optionObj.strike, 0)
        elif optionType == 'put':
            payoff = np.maximum(self.optionObj.strike - samplePaths[:,-1], 0)
        else:
            raise ValueError("Invalid option type. Choose 'call' or 'put'.")
            
        #dicount the payoff 
        payoff = payoff * np.exp(-self.stockObj.r * self.optionObj.timeToExpiry)
        
        return np.mean(payoff)
        
        
    

In [39]:
class AmOption():

    def __init__(self, stockObj, optionObj, modelObj):
        self.stockObj = stockObj
        self.optionObj = optionObj
        self.modelObj = modelObj
        
    def payOff(self, pathCrossSection):
        return np.maximum(self.optionObj.strike - pathCrossSection, 0)
    
    def getDt(self):
        return self.optionObj.timeToExpiry/self.optionObj.nSteps
    
    def regressionFunction(self, pathCrossSectionPayOff, pathCrossSectionTminus1, exerciseD, dfs):
        #calculate dt
        dt = self.getDt()
        #create empty dataframe to hold Y and X
        regression = pd.DataFrame()
        #print(pathCrossSectionPayOff * exerciseD * dfs)
        regression['Y'] = np.sum(pathCrossSectionPayOff * exerciseD * dfs, axis = 1)
        #print(pathCrossSectionTminus1[:, 0])
        regression['X'] = pathCrossSectionTminus1[:, 0]
        #find where option is in the money at T-1 and include only those paths in regression
        regression = regression[regression['X'] < self.optionObj.strike]
        # Create 'X^2' (squared term)
        regression.loc[:, 'X_Squared'] = regression['X'] ** 2
        # Define the independent variables (including constant)
        X = regression[['X', 'X_Squared']]
        X = sm.add_constant(X)  # Adds the constant term to the model

        # Define the dependent variable
        y = regression['Y']
        
        # Perform the regression
        model = sm.OLS(y, X).fit()
        
        return model, regression
    
    def getContinuationValue(self, regFunc, pathCrossSectionTminus1, immediatePayoff):
        #get parameters of the regFunc
        intercept = regFunc.params['const']
        coefficient_X = regFunc.params['X']
        coefficient_X_squared = regFunc.params['X_Squared']
        #calculate continuation value
        return intercept + coefficient_X * pathCrossSectionTminus1 + coefficient_X_squared * (pathCrossSectionTminus1 ** 2)
        
        
    def priceAmByLSMC(self, samplePaths, dt):
        #create an empty matrix exercise decision 0 - no exercise, 1 - exercise
        exerciseDecisionMatrix = np.zeros((self.modelObj.numberOfPaths, self.modelObj.nSteps))
        #similary create empty payoff matrix
        payoffMatrix = np.zeros((self.modelObj.numberOfPaths, self.modelObj.nSteps))
        
        #get the payoff at the end
        payoffMatrix[:, -1] = self.payOff(samplePaths[:, -1])
        exerciseDecisionMatrix[:, -1] = np.where(payoffMatrix[:, -1] > 0 , 1, 0)
        #discount factors
        discountFactors = np.array([])
        #print(payoffMatrix)
        #print(exerciseDecisionMatrix)
        #loop throgh all time points and run LSM
        results_dict = {}
        for t in range(self.modelObj.nSteps - 2, -1, -1):
            
            #get immmediate payoff
            payoffMatrix[:, t] = self.payOff(samplePaths[:, t+1])
            exerciseDecisionMatrix[:, t] = np.where(payoffMatrix[:, t] > 0, 1 , 0)
            #calculate dicount factors
            discountFactor = np.exp(-self.modelObj.stockObj.r * dt * (self.modelObj.nSteps  - t - 1))
            #print("self.modelObj.nSteps  - t - 1 : ", self.modelObj.nSteps  - t - 1, "discountFactor : ", discountFactor )
            discountFactors = np.append(discountFactors, discountFactor)
            #print("t : ", t, " ", discountFactors)
            
            #get regression function to calculate conditional expectation
            #print(samplePaths[:, t+1:].shape)
            regFunction, regDF = self.regressionFunction(payoffMatrix[:, t+1:], samplePaths[:, t+1:], exerciseDecisionMatrix[:, t+1:], discountFactors)
            #pd.DataFrame(payoffMatrix[:, t+1:]).to_csv("payoffMatrix_" + str(t+1) + '.csv')
            #pd.DataFrame(samplePaths[:, t+1:]).to_csv("samplePaths_" + str(t+1) + '.csv')
            #pd.DataFrame(exerciseDecisionMatrix[:, t+1:]).to_csv("exerciseDecisionMatrix" + str(t+1) + '.csv')
            # Store relevant results in a dictionary
            results_dict[f'Regression_{t}'] = {
            'coefficients': regFunction.params,
            }
            
            #print(regFunction.params)
            #get continuation decision
            contValue = self.getContinuationValue(regFunction, samplePaths[:, t+1], payoffMatrix[:, t])
            #regDF['contValue'] = contValue
            #check if contValue is greater than immediate payoff and replace that
            condition = (payoffMatrix[:, t] > contValue) & (payoffMatrix[:, t] > 0)
            #regDF['condition'] = condition
            payoffMatrix[:, t] = np.where(condition, payoffMatrix[:, t], 0)
            exerciseDecisionMatrix[:, t] = np.where(condition, 1 , 0)
            #regDF['exerciseDecision'] = exerciseDecisionMatrix[:, t]
            #also make sure all the payoffs after exersice are zero
            payoffMatrix[condition, t+1:] = 0
            #print(condition)
            exerciseDecisionMatrix[condition, t+1:] = 0
            #print(payoffMatrix)
            #print(exerciseDecisionMatrix)
            #regDF.to_csv('regDF_' + str(t) + '.csv')
        #caculate final price
        #dicount all payofss
        #print(discountFactors)
        for i in range(0, self.optionObj.nSteps):
            payoffMatrix[:,i] = payoffMatrix[:,i] * np.exp(-self.stockObj.r * dt * (i+1))
        #take mean
        V = np.sum(payoffMatrix) / self.modelObj.numberOfPaths
            
        return V, payoffMatrix, exerciseDecisionMatrix, results_dict
        
    

In [57]:
# s0 = 40
# sigma = 0.4
# r = 0.06
# paths = 50000
# T = 2
# steps = T * 50
# K = 40

# #create stock with spot, vol and RFR
# stock1 = Stock(s0, sigma, r)
# #create option object
# Option1 = Option(T, steps, K)
# #create BS model object with required number of paths and steps for above stock
# Model = BSModel(stock1, Option1, paths, steps)
# #create option now
# EurOption1 = EurOption(stock1, Option1)
# #generate random variables
# randomNums = Model.generateRandonNumbers(0)
# #generate paths now
# #simulationPaths = Model.generateSamplePaths(randomNums)
# # simulationPaths = pd.read_csv('simulated_paths.csv')
# # simulationPaths = simulationPaths.to_numpy()
# simulationPaths = Model.generateSamplePaths(randomNums)
# simulationPathsAnt = Model.generateSamplePaths(-randomNums)

# #final price average of two
# eurPutOptPriceByMC =  round((EurOption1.priceByBSMC(simulationPaths, "put") + EurOption1.priceByBSMC(simulationPathsAnt, "put")) / 2, 3)
# eurPutOptPriceByAnalytic = round(EurOption1.priceByBSAnalytic("put"), 3)



# print("Price by Black-Scholes analytic formuala of a put option : " , eurPutOptPriceByAnalytic)
# print("Price by Black-Scholes Monte-Carlo simulation of a put option : " , eurPutOptPriceByMC)


Price by Black-Scholes analytic formuala of a put option :  6.326
Price by Black-Scholes Monte-Carlo simulation of a put option :  6.326


In [58]:
# # Data
# k = 1.1
# r = 0.06
# samplePaths = np.array(
#     [[1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
#      [1.09, 1.16, 1.22, 0.93, 1.11, 0.76, 0.92, 0.88],
#      [1.08, 1.26, 1.07, 0.97, 1.56, 0.77, 0.84, 1.22],
#      [1.34, 1.54, 1.03, 0.92, 1.52, 0.90, 1.01, 1.34]])


    
# s0 = 1.00
# sigma = 0.4
# r = 0.06
# paths = 8
# steps = 3
# T = 3
# K = 1.1

# #create stock with spot, vol and RFR
# stock2 = Stock(s0, sigma, r)
# #create option object
# Option2 = Option(T, steps, K)
# #create BS model object with required number of paths and steps for above stock
# Model = BSModel(stock2, Option2, paths, steps)


In [59]:
# #create american option now
# AmOption2 = AmOption(stock2, Option2, Model)
# dt = AmOption2.getDt()
# V, payoffMatrix, exerciseDecisionMatrix = AmOption2.priceAmByLSMC(samplePaths.T, dt)
# print(V)

In [64]:
# #create american option now
# AmOption1 = AmOption(stock1, Option1, Model)
# dt = AmOption1.getDt()
# print(dt)
# V, payoffMatrix, exerciseDecisionMatrix, results_dict = AmOption1.priceAmByLSMC(simulationPaths, dt)
# V1, payoffMatrix1, exerciseDecisionMatrix1, results_dict1 = AmOption1.priceAmByLSMC(simulationPathsAnt, dt)
# amOptionPrice = (V+V1)/2
# print(amOptionPrice)

0.02
6.903432584715462


In [61]:
#np.savetxt("simulationPaths.csv", simulationPaths, delimiter=",")

In [62]:
#np.savetxt("randomNums.csv", randomNums, delimiter=",")

In [65]:
# print(round(V1, 4))

6.9417


In [73]:
# arr = np.append(np.sum(payoffMatrix, axis = 1),(np.sum(payoffMatrix1, axis = 1)))
# np.std(arr, ddof=1) / np.sqrt(len(arr))

0.02186953507462002

In [38]:
# for key in results_dict:
#     print(key)
#     print(results_dict[key])
    

Regression_98
{'coefficients': const        40.875878
X            -1.066515
X_Squared     0.001165
dtype: float64}
Regression_97
{'coefficients': const        41.561030
X            -1.117680
X_Squared     0.002082
dtype: float64}
